In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

#from Database import Databases
#from DownloadHTML import DownloadHTML
#from AdressCollecting import *
import urllib.request as req
import re
import time
import os
import ssl
from selenium.common.exceptions import NoSuchElementException, NoSuchAttributeException, TimeoutException, InvalidSessionIdException
from bs4 import BeautifulSoup as bs
import requests

In [2]:
url = "https://namu.wiki/w/%EC%88%98%EC%88%98%EA%BD%83%EB%8B%A4%EB%A6%AC"

response = requests.get(url)

soup = bs(response.text, 'html.parser')

In [3]:
ptime = time.time()
response = requests.get("https://namu.wiki/w/%EA%B4%80%EC%82%AC(%ED%92%88%EC%82%AC)")
print(time.time()-ptime)
ptime=time.time()
soup = bs(response.text, 'html.parser')
print(time.time()-ptime)
ptime=time.time()
name = soup.find_all('title')[0]
print(time.time()-ptime)
ptime=time.time()
name = name.text.replace(' - 나무위키', '')

0.4274616241455078
0.06555438041687012
0.001001596450805664


In [20]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import string
import random
import requests
from bs4 import BeautifulSoup as bs

from urllib.parse import quote_plus
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

#from Database import Databases
#from DownloadHTML import DownloadHTML
#from AdressCollecting import *
import urllib.request as req
import re
import time
import os
import ssl
from selenium.common.exceptions import NoSuchElementException, NoSuchAttributeException, TimeoutException, InvalidSessionIdException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import trange
import random

def init_driver():
    
    # 인증서 문제 해결
    ssl._create_default_https_context = ssl._create_unverified_context
    
    options = Options()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument('--no-sandbox')
    options.add_argument("disable-gpu")
    options.add_argument('window-size=1920x1080')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    
    driver = None

    my_driver = webdriver.Chrome(options = options)
    my_driver.set_window_position(0, 0)
    # driver.maximize_window()
    my_driver.set_window_size("1000", "1000")
    my_driver.set_page_load_timeout(10)
    return my_driver

class namu_crawler():
    def __init__(self, init_url, save_dir='data', link_dir='links'):
        self.init_url = ""
        self.global_link = pd.DataFrame(columns = ['Name', 'Link'])

        self.link_dir = link_dir
        self.link_files = []
        
        for f in os.listdir('./'):
            if "links" in f:
                self.link_files.append(f)
                
        if len(self.link_files)>0:
            self.global_link = pd.read_csv(self.link_files[-1], index_col=0)
        else:
            self.init_global_link()
            
        self.save_dir = save_dir
        
    def init_global_link(self):
        url = "https://namu.wiki"+self.init_url
        response = requests.get(url)
        soup = bs(response.text, 'html.parser')
        name = soup.find_all('title')[0].text.replace(' - 나무위키', '')
        self.global_link = pd.DataFrame([(name, self.init_url)], columns = ['Name', 'Link'])
        
    def get_link(self, url, index):
        response = requests.get(url)
        soup = bs(response.text, 'html.parser')
        try:
            text = soup.find('title').text
            test = text=="비정상적인 트래픽 감지"
        except:
            test = False
        
        def custom_wait(driver, by, value):
            element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((by, value))
            )
            
            while True:
                try:
                    element = WebDriverWait(driver, 10).until(
                        EC.invisibility_of_element_located((by, value))
                    )
                    if element:
                        return element
                except Exception as e:
                    # Handle exceptions if needed, or just continue waiting
                    pass
                    
        if test:
            driver = init_driver()
            while text == "비정상적인 트래픽 감지":
                driver.get(url)
                custom_wait(driver, By.XPATH, '/html/body/section/form/input[2]')
                response = requests.get(url)
                soup = bs(response.text, 'html.parser')
                text = soup.find('title').text
            driver.quit()
            time.sleep(60*10)
            
        with open(self.save_dir+'/'+('%015d' % index)+'.txt', 'w', encoding='utf8') as f:
            f.write(str(soup))
            
        links = soup.find_all('a', 'wiki-link-internal')
        name = []
        internal_link = []
        for link in links:
            decoded_name = urllib.parse.unquote(link.get_text())  # URL 디코딩
            decoded_href = urllib.parse.unquote(link.attrs['href'])
            internal_link.append((decoded_name, decoded_href))
            
        return internal_link
        
    def check_file(self):
        """
        names = os.listdir(self.save_dir)
        if names!=[]:
            return int(names[-1].split('.')[0])
        """
        if len(self.link_files)>=2:
            return len(pd.read_csv(self.link_files[-2], index_col=0))
            
        return 0
        
    def start_crawl(self, step = 1, url=""):
        internal_link = self.global_link.loc[self.check_file():]
        print("Starting from ....")
        print(internal_link)
        #return 
        for i in range(len(self.link_files), step+len(self.link_files)):
            new_link = pd.DataFrame(columns = ['Name', 'Link', 'Key'])
            for j, link in tqdm(internal_link.iterrows(), total=internal_link.shape[0]):
                ptime=time.time()
                new_url = "https://namu.wiki"+link['Link']
                new_links_ = self.get_link(new_url, j)
                new_link_ = pd.DataFrame(new_links_, columns = ['Name', 'Link'])
                merged_df = pd.concat([new_link, new_link_])
                new_link = merged_df.drop_duplicates(subset=['Link'], keep="first")
                sleep_time = 1-(time.time()-ptime)
                sleep_time = sleep_time if sleep_time>0 else 0
                time.sleep(sleep_time+random.uniform(0, 0.5))
                
                
            #self.global_link.to_csv(f'links_{i}.csv')
                
                #print(new_link)
                #keys = make_keys(global_link['Key'], len(new_link))
            length = len(self.global_link)
            self.global_link = pd.concat([self.global_link, new_link])
            self.global_link = self.global_link.drop_duplicates(subset=['Link'], keep="first")
            self.global_link = self.global_link.reset_index().drop(columns=['index'])
            #print(global_link)
            self.global_link.to_csv(f'links_{i}.csv', encoding='utf-8-sig')
            internal_link = self.global_link.loc[length:].copy()
        #return global_link

In [21]:
crawler = namu_crawler("/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EC%A3%BC%EC%9A%94%20%ED%8E%98%EC%9D%B4%EC%A7%80%20%EB%A7%81%ED%81%AC")

In [23]:
import os
import urllib

# 디렉토리가 존재하지 않는 경우 생성
if not os.path.exists('data'):
    os.makedirs('data')

# 이후 크롤링 코드 실행
#time.sleep(60*10)
crawler.start_crawl(4) 

Starting from ....
      Name Link
0  나무위키:대문     


 14%|█▍        | 11/78 [00:14<01:25,  1.27s/it]

In [6]:
url = "https://namu.wiki/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EC%A0%91%EA%B7%BC%20%EC%A0%9C%ED%95%9C/%EB%AC%B8%EC%84%9C%20%EB%AA%A9%EB%A1%9D/4%EB%8B%A8%EA%B3%84/2021%EB%85%84%20%EC%9D%B4%EC%A0%84#s-"

In [7]:

response = requests.get(url)
soup = bs(response.text, 'html.parser')
soup


<!DOCTYPE html>

<html data-n-head-ssr=""><head><title>나무위키:접근 제한/문서 목록/4단계/2021년 이전 - 나무위키</title><meta charset="utf-8" data-n-head="ssr"/><meta content="user-scalable=no, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0, width=device-width" data-n-head="ssr" name="viewport"/><meta content="ie=edge" data-n-head="ssr" http-equv="x-ua-compatible"/><meta content="the seed" data-n-head="ssr" name="generator"/><meta content="yes" data-n-head="ssr" name="mobile-web-app-capable"/><meta content="나무위키" data-n-head="ssr" name="application-name"/><meta content="나무위키" data-n-head="ssr" name="msapplication-tooltip"/><meta content="/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EB%8C%80%EB%AC%B8" data-n-head="ssr" name="msapplication-starturl"/><meta content="max-image-preview:large" data-n-head="ssr" name="robots"/><meta content="#008275" data-n-head="ssr" name="theme-color"/><meta content="noarchive" data-n-head="ssr" name="googlebot"/><link data-n-head="ssr" href="https://namu.wiki/w/%EB%82%98%

NameError: name 'driver' is not defined

NameError: name 'driver' is not defined

In [12]:
a = pd.DataFrame([(1, 2, 3), (1, 2, 3)], columns = ['a', 'b', 'c'])
b = pd.DataFrame([(1, 2, 3), (1, 2, 3)], columns = ['a', 'b', 'c'])
a = pd.concat([a, b])-
a.insert(2, "Keys", make_keys(a['c'], len(a['c'])), True)
a

NameError: name 'make_keys' is not defined

In [13]:
b = a.reset_index().drop(columns = ['index'])
b.loc[1:]

a  b  c
1  1  2  3
2  1  2  3
3  1  2  3

In [306]:
c = b.reset_index()
c

index  a  b        Keys  c
0      0  1  2  Q5B5BEOBGF  3
1      1  1  2  DKWF68VGCS  3
2      2  1  2  31CABP2T0H  3
3      3  1  2  LDKAL7W0IG  3

In [307]:
start_crawl(6)

  2%|█▋                                                                    | 17193/723014 [1:50:46<75:47:46,  2.59it/s]


KeyboardInterrupt: 

In [ ]:
a = pd.DataFrame([(1, 2), (2, 3)], columns = ['a', 'b'])
b = pd.DataFrame([(1, 2), (3, 4)], columns = ['a', 'b'])

In [81]:
for i, t in a.iterrows():
    print(t['b'])

2
3


In [113]:
quote_plus("%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EA%B8%B0%EB%B3%B8%EB%B0%A9%EC%B9%A8/%EB%AC%B8%EC%84%9C%20%EA%B4%80%EB%A6%AC%20%EB%B0%A9%EC%B9%A8")


'%25EB%2582%2598%25EB%25AC%25B4%25EC%259C%2584%25ED%2582%25A4%3A%25EA%25B8%25B0%25EB%25B3%25B8%25EB%25B0%25A9%25EC%25B9%25A8%2F%25EB%25AC%25B8%25EC%2584%259C%2520%25EA%25B4%2580%25EB%25A6%25AC%2520%25EB%25B0%25A9%25EC%25B9%25A8'

In [ ]:
https://namu.wiki#i1-s-.txt

In [63]:
import pandas as pd

# Create two DataFrames with the same column names
df1 = pd.DataFrame({'ID': [1, 2, 3], 'Name': ['Alice', 'Bob', 'Charlie']})
df2 = pd.DataFrame({'ID': [4, 5, 6], 'Name': ['David', 'Eva', 'Frank']})

# Merge DataFrames based on the common column 'ID'
merged_df = pd.merge(df1, df2, on='ID', how='inner')

# Print the merged DataFrame
print(merged_df)


Empty DataFrame
Columns: [ID, Name_x, Name_y]
Index: []


In [67]:
pd.merge(df1, df2, on='ID', how='inner')

Empty DataFrame
Columns: [ID, Name_x, Name_y]
Index: []